<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Specific-Aims" data-toc-modified-id="Specific-Aims-1">Specific Aims</a></span></li><li><span><a href="#Citations" data-toc-modified-id="Citations-2">Citations</a></span></li></ul></div>

# An Investigation of Keyhole Limpet Hemocyanin

<img src="Images/Keyhole Limpet Nigel Phillips IMG_1389.JPG"><br>Source: Nigel Phillips[[1]](https://www.wildlifetrusts.org/wildlife-explorer/marine/sea-snails-and-sea-slugs/keyhole-limpet)

## Specific Aims

Hemocyanin is the copper-containing, main oxygen carrying protein commonly used in the blood of arthropods, mollusks, and some other invertebrates. Hemocyanin is of interest in biomedical research because of its use as an adjuvant in vaccine development. Unfortunately, current production of hemocyanin as an adjuvant is extremely expensive with the harvesting process requiring blood from the limpet Megathura crenulata. Key-Hole Limpet hemocyanin (KLH) is challenging to produce with recombinant techniques due to the large size of the protein. The cost of extraction ranges between five thousand to one hundred and fifty thousand dollars per gram of KLH. <br><br>  
Due to the increased demand, vaccine adjuvants may present as a bottleneck for future vaccine development. Recombinant KLH that has been produced with E. coli, however, is not used for general medical practice due to unsuitable purity. While the structure of KLH has been resolved to 9 Angstroms (A), with its functional unit resolved to 4A, potential alternative hemocyanin sources to this protein have not been determined with imaging techniques such as Cryo-EM or X-ray crystallography.  <br><br>
The goal of this work will be two-fold.  First, this work aims to understand the significance of adjuvants in relation to KLH, exploring the properties of its medicinal use and tradeoffs to alternative adjuvants. Second, this work aims to explore the structure of KLH using computational and bioinformatic techniques in conjunction with analysis found in literature, highlighting potential alternatives to this important protein. 


## Citations

[[1]](https://www.wildlifetrusts.org/wildlife-explorer/marine/sea-snails-and-sea-slugs/keyhole-limpet) www.wildlifetrusts.org